In [1]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb
import gmso
import foyer
from foyer import Forcefield

from gmso.external.convert_mbuild import to_mbuild
from foyer import general_forcefield

import unyt as u

Quickly set up an MPC monomer from a SMILES string without the Si initiator

In [2]:
mpc = mb.load("CC(=C)C(=O)OCCOP([O-])(=O)OCC[N+](C)(C)C", smiles=True)

In [3]:
mpc.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Which parameters are missing?

First however, let us run against the main oplsaa file to identify which angles/dihedrals were missing.  The output is copied and pasted below.  These missing parameters have been added into the mpc.xml file based upon the choices made by ligpargen (which mostly appear to be using parameters for related groupings that exist in the main opls file). 

```
Missing angle with ids (1, 3, 5) and types ['opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (0, 1, 3, 4) and types ['opls_135', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (0, 1, 3, 5) and types ['opls_135', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (1, 3, 5, 6) and types ['opls_141', 'opls_465', 'opls_467', 'opls_182'].
Missing dihedral with ids (2, 1, 3, 4) and types ['opls_143', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (2, 1, 3, 5) and types ['opls_143', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (3, 1, 0, 19) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (3, 1, 0, 20) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (3, 1, 0, 21) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
```


In [4]:
oplsaa = Forcefield(forcefield_files="oplsaa_imodels.xml")
mpc_test = oplsaa.apply(mpc, verbose=True,
                          assert_angle_params=False,
                          assert_dihedral_params=False)

Missing angle with ids (1, 3, 5) and types ['opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (0, 1, 3, 4) and types ['opls_135', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (0, 1, 3, 5) and types ['opls_135', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (1, 3, 5, 6) and types ['opls_141', 'opls_465', 'opls_467', 'opls_182'].
Missing dihedral with ids (2, 1, 3, 4) and types ['opls_143', 'opls_141', 'opls_465', 'opls_466'].
Missing dihedral with ids (2, 1, 3, 5) and types ['opls_143', 'opls_141', 'opls_465', 'opls_467'].
Missing dihedral with ids (3, 1, 0, 19) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (3, 1, 0, 20) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].
Missing dihedral with ids (3, 1, 0, 21) and types ['opls_465', 'opls_141', 'opls_135', 'opls_140'].


### So how do we get these parameters?  


There are two basic approaches:

* 1 Make "reasonable" choices, subbing in related existing parameters. This is the common approach used by many people in OPLS (including ligpargen).  As a completely made up example, if we were missing a dihedral for CM-CT-CT-O (basically 3 carbons and an oxygen) we might make the choice to use CT-CT-CT-O.  

* 2 Parameterize the interaction. This would rely upon performing QM calculations to get the energy landscape as a function of phi (or theta for an angle).  For certain groups this is absolutely essential (see our work on perfluoropolyethers), because existing parameters would in no way work.  This used to be a really expensive operation, but QM calculations are no longer absurdly expensive.  I've been trying to carve out some time to work on an automated workflow for this; a lot of the dihedrals in OPLS are not very good (because these calculations used to be very expensive). 

For the case here, we will just do (1) and make reasonable choices.  In general, the bonds/angles/dihedrals in opls as assigned based on the "class" (sometimes call family).  Basically, this is based on the idea that the dihedral parameters are more general than atomtypes.  So you might have a whole bunch of different atom_types for carbon that all are part of "class" CT, so you need fewer bonds/angles/dihedrals defined. 

So the parameters I added to "mpc.xml" I did this simply by looking for related parameters and then comparing against the choices made by ligpargen.  

In [5]:
oplsaa_mpc_old = Forcefield(forcefield_files="mpc.xml")

In [6]:
typed_chain_old = oplsaa_mpc_old.apply(mpc, verbose=True,
                          assert_bond_params=False,
                          assert_angle_params=False,
                          assert_dihedral_params=False)

We now have a working xml file that will fully atom type. 

A few notes:

The new version of Foyer that uses GMSO has a lot more features and will soon be replacing the old verison.  However, it is currently slower (speedups are a WIP!) and there is a bug with reading impropers from a file that I'm going to need to look into. 

For the sake of current lack of documentation with some of the features, I've just done the same thing with the new version, just with impropers commented out of the xml file.   You'll see it throws a warning about the missing improper.

In [7]:
oplsaa_mpc_new = general_forcefield.Forcefield(forcefield_files='mpc_no_impropers.xml', strict=False)

In [8]:
typed_chain_new = oplsaa_mpc_new.apply(mpc, assert_improper_params=False, assert_dihedral_params=False, debug=True)

Improper with missing parameters: 
{'Improper': ['opls_288', 'opls_292', 'opls_291', 'opls_291']}


As a quick check I compared against the number of bonds/angles/dihedrals in the ligpargen file and get the same numnbers (note, they included impropers in the same section...most have a value of 0 given to them, aside from the one we were missing above). 

In [9]:
print('n bonds: ', typed_chain_new.n_bonds)
print('n angles: ', typed_chain_new.n_angles)
print('n proper dihedrals:',  typed_chain_new.n_dihedrals)

n bonds:  40
n angles:  72
n proper dihedrals: 85


### Charges

We already know this isn't going to be charge neutral, but let us loop through and take a look as well as printing out the SMARTS string to put a bunch of relevant info all in one spot.  

I'll note old and new foyer are basically same, but the new version uses unyt for units (rather than the built in units class in parmed).  As such, in new foyer, we can tack on '.v' on charge to have it return the value without the messy unit label.    

...new foyer also calls things 'sites' rather than 'atoms' in the datastructure. 

In any case, it is good to see that new and old Foyer produce the same output given the same effective inputs.

In [10]:
sum_of_charges = 0
for atom in typed_chain_new.sites:
    print(atom.name, '\t', atom.atom_type.name,  '\t', atom.charge.v, '\t', oplsaa_mpc_new.atomTypeDefinitions[atom.atom_type.name])
    sum_of_charges += atom.atom_type.charge
print(f"Total charge of molecule {sum_of_charges.v}")

C 	 opls_135 	 -0.18 	 [C;X4](C)(H)(H)H
C 	 opls_141 	 0.0 	 [C;X3](C)(C)C
C 	 opls_143 	 -0.23 	 [C;X3](C)(H)H
C 	 opls_465 	 0.51 	 [C;X3]([O;X1])([O;X2])
O 	 opls_466 	 -0.43 	 [O;X1]([C;%opls_465]([O;%opls_467]))
O 	 opls_467 	 -0.33 	 [O;X2]([C;%opls_465])([!H])
C 	 opls_182 	 0.14 	 [C;X4]([O;%opls_180])(H)(H)
C 	 opls_443 	 0.3 	 [C;X4]([O;%opls_442])(C)(H)H
O 	 opls_442 	 -0.6 	 [O;X2]([P;%opls_440])([C;X4])
P 	 opls_440 	 1.62 	 [P;X4]([O;X1])([O;X1])([O;X2][C;X4])([O;X2][C;X4])
O 	 opls_441 	 -0.92 	 [O;X1]([P;%opls_440])
O 	 opls_441 	 -0.92 	 [O;X1]([P;%opls_440])
O 	 opls_442 	 -0.6 	 [O;X2]([P;%opls_440])([C;X4])
C 	 opls_443 	 0.3 	 [C;X4]([O;%opls_442])(C)(H)H
C 	 opls_292 	 0.19 	 [C;X4](H)(H)([C;X4])([N;%opls_288])
N 	 opls_288 	 0.0 	 [N;X4](C)(C)(C)C
C 	 opls_291 	 0.13 	 [C;X4](H)(H)(H)([N;%opls_288])
C 	 opls_291 	 0.13 	 [C;X4](H)(H)(H)([N;%opls_288])
C 	 opls_291 	 0.13 	 [C;X4](H)(H)(H)([N;%opls_288])
H 	 opls_140 	 0.06 	 H[C;X4]
H 	 opls_140 	 0.06 	 H[C;X4]


In [11]:
sum_of_charges = 0
for i, atom in enumerate(typed_chain_old.atoms):
    print(i, '\t', atom.name, '\t', atom.atom_type.name,  '\t', atom.charge, '\t', oplsaa_mpc_old.atomTypeDefinitions[atom.atom_type.name])
    sum_of_charges += atom.charge
print(f"Total charge of molecule {sum_of_charges}")

0 	 C 	 opls_135 	 -0.18 	 [C;X4](C)(H)(H)H
1 	 C 	 opls_141 	 0.0 	 [C;X3](C)(C)C
2 	 C 	 opls_143 	 -0.23 	 [C;X3](C)(H)H
3 	 C 	 opls_465 	 0.51 	 [C;X3]([O;X1])([O;X2])
4 	 O 	 opls_466 	 -0.43 	 [O;X1]([C;%opls_465]([O;%opls_467]))
5 	 O 	 opls_467 	 -0.33 	 [O;X2]([C;%opls_465])([!H])
6 	 C 	 opls_182 	 0.14 	 [C;X4]([O;%opls_180])(H)(H)
7 	 C 	 opls_443 	 0.3 	 [C;X4]([O;%opls_442])(C)(H)H
8 	 O 	 opls_442 	 -0.6 	 [O;X2]([P;%opls_440])([C;X4])
9 	 P 	 opls_440 	 1.62 	 [P;X4]([O;X1])([O;X1])([O;X2][C;X4])([O;X2][C;X4])
10 	 O 	 opls_441 	 -0.92 	 [O;X1]([P;%opls_440])
11 	 O 	 opls_441 	 -0.92 	 [O;X1]([P;%opls_440])
12 	 O 	 opls_442 	 -0.6 	 [O;X2]([P;%opls_440])([C;X4])
13 	 C 	 opls_443 	 0.3 	 [C;X4]([O;%opls_442])(C)(H)H
14 	 C 	 opls_292 	 0.19 	 [C;X4](H)(H)([C;X4])([N;%opls_288])
15 	 N 	 opls_288 	 0.0 	 [N;X4](C)(C)(C)C
16 	 C 	 opls_291 	 0.13 	 [C;X4](H)(H)(H)([N;%opls_288])
17 	 C 	 opls_291 	 0.13 	 [C;X4](H)(H)(H)([N;%opls_288])
18 	 C 	 opls_291 	 0.13 	 [C;X4]

Cal put in some routines in GMSO to allow you to visualize the bondgraph and interact with it.  He did this while trying to figure out missing parameters as well. 

There is still some work that needs to be done to make it easier to read, allow you to adjust sizes, etc. It shouldn't really take much work I don't think to make it actually usefully (mostly just need to change the plotting area to be larger I think).  At this point not super helpful, but I figured I'd include it.  


In [12]:
from gmso.formats.networkx import interactive_networkx_atomtypes


interactive_networkx_atomtypes(typed_chain_new)

/Users/cri/opt/anaconda3/envs/perm/lib/python3.9/site-packages/ipykernel/pylab/backend_inline.py:11: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  warnings.warn(


interactive(children=(Dropdown(description='Label', options=('atom_type.name', 'charge', 'mass', 'element', 'l…

Cal also put together some really helpful code that uses plotly and lets you interact with the graph, selecting things, moving them around, etc. 

This I think is really useful when trying to look at how charge is distributed in the system and how to apply other work.   I'll note I added the charge to the node labels...for some reason they are showing up as negative of the actual value...something must be off in the "demo_utils.py" script (will need to check into that later). 


In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_cytoscape as cyto
import dash_table
import unyt
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

from dash.dependencies import Input, Output
import json
from jupyter_dash import JupyterDash
import molecule_parts
import demo_utils
from gmso.external.convert_networkx import to_networkx



In [14]:
graph = to_networkx(typed_chain_new)
topology = typed_chain_new


In [ ]:
#Dash Plotly representation of the molecule
app = JupyterDash(__name__)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

color_dictionary = {'"C"':"#8C92AC",'"O"':"red",'"H"':"#848482",'"N"':"blue",'"Cl"':'green'}
color_selection = []
for key,value in color_dictionary.items():
    color_selection.append({'selector': "[element = " +  str(key) + "]",
                            'style': {'background-color': str(value),'shape': 'circle'}})

layout = nx.drawing.layout.kamada_kawai_layout(graph)
elements2 = []
nodeids = {}
for i,node in enumerate(graph.nodes):
    elements2.append({'data': {'id': str(i) + ': ' + node.name, 
                               'label': node.name + '(' + f'{node.charge.v}' + ')',
                              'element': node.name,
                              'hash': node.__hash__(),
                              'index' : i},
                     'classes': '',
                     'position': {'x': layout[node][0]*500,'y': layout[node][1]*500},
                     'size': 200})
    nodeids[node] = str(i) + ': ' + node.name

for edge in graph.edges:
    elements2.append({'data': {'source': nodeids[edge[0]], 'target': nodeids[edge[1]]}, 'classes': 'red'})
       
app.layout = html.Div([
    html.P("Dash Cytoscape:"),
    dcc.Dropdown(
        id='parameter-dropdown',
        options=[
            {'label': 'Atom Types', 'value': 'atom_type'},
            {'label': 'Bond Types', 'value': 'bond_type'},
            {'label': 'Angle Types', 'value': 'angle_type'},
            {'label': 'Dihedral Types', 'value': 'dihedral_type'},

        ],
        value='atom_type'),
    html.Div([
        cyto.Cytoscape(
            id='cytoscape',
            elements= elements2,
            style={'width': '900px', 'height': '500px'},
            layout={'name':'preset'},
            stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)', 'width': "50%", 'height': "50%"
                }
            }
        ] + color_selection 
        )],
    style={'width': '45%', 'display': 'inline-block', 'margin-right': '20px',
          'borderBottom': 'thin lightgrey solid',
          'borderTop': 'thin lightgrey solid',
          'backgroundColor': 'rgb(250, 250, 250)',
          'vertical-align': 'top'}
    ),    
    html.Div([    
        dash_table.DataTable(
            id='table-of-parameters',
            style_as_list_view=True,
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto',
                'lineHeight': '15px'
            },
            style_table={'overflowX': 'auto','overflowY': 'auto',
                        'height': '500px', 'width': '400px'},
            style_data_conditional=[
            {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(248, 248, 248)'
            }],
            style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
            }
        )
    ],
    style={'width': '45%','display': 'inline-block', 'padding': '0 0'}
    )
])

@app.callback(
    Output('table-of-parameters', 'columns'),
    Output('table-of-parameters', 'data'),
    Input('parameter-dropdown','value'))
def update_datatable(input_value):
    if input_value == 'atom_type':
        df = demo_utils.atomtypes_to_datatables(graph,
                                                labels=None,atom_objects = True)
    elif input_value == 'bond_type':
        df = demo_utils.bondtypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'angle_type':
        df = demo_utils.angletypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'dihedral_type':
        df = demo_utils.dihedraltypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    return([{"name": i, "id": i} for i in df.columns],
          df.to_dict('records'))

@app.callback(Output('table-of-parameters', 'style_data_conditional'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    try:
        return ([{
            'if': {'filter_query': '{atom_id} = ' + str(data['hash'])},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']),
                   'column_id': ''},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) +
                                ' || {atom4_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'}

        ])
    except TypeError:
        return


app.run_server(mode = "inline")

The charges christoph used initially seem to be based off of this paper (but was failed to be appropriately cited in the main text...probably be cause the time from starting a paper to finishing is so damn long and we didn't have a great system yet for keeping track of all of this): dx.doi.org/10.1021/jp5016627

```
; Refined OPLS-AA force field for saturated phosphatidylcholine bilayers at full hydration
; A. Maciejewski, M. Pasenkiewicz-Gierula, I. Vattulainen, T. Rog 

;
; Refined OPLS-AA force field for saturated phosphatidylcholine bilayers at full hydration
; A. Maciejewski, M. Pasenkiewicz-Gierula, I. Vattulainen, T. Rog 
;
;       2
;       |
;   7 4-1-3   11
;    \  |    /    
;   8-5-13-9-12 
;    /  |    \   
;  6 15-14-16 10
;       |                               1  1  1  1  1  1  1  1  
;   19-17-18             90   89 96 95 02 01 08 07 14 13 20 19 125 126
;       |                  \  /  \  /  \  /  \  /  \  /  \  /  \  /
;       20   21       87    88    94   100    106   112   118   124  130
;        \  /           \  /  \  /  \  /  \  /  \  /  \  /  \  /  \  /
;         23  80   81 84 85    91    97    103   109   115   121   127
;        / \   \  /   ||/  \  /  \  /  \ 1/ 1\ 1/ 1\ 1/ 1\ 1/ 1\  /  \ 
;       24  22  79-82-83  86 93 92 99 98 05 04 11 10 17 16 23 22 129  128
;        \     /                 
;         25-28-29 34  35 40 41 46 47 52 53 58 59 64 65 70 71 76  77
;        /  \  \     \  /  \  /  \  /  \  /  \  /  \  /  \  /  \  /
;       27  26  30-31-33    39    45    51    57    63    69    75
;                  ||   \  /  \  /  \  /  \  /  \  /  \  /  \  /  \
;                  32    36    42    48    54    60    66    72    78
;                       /  \  /  \  /  \  /  \  /  \  /  \  /  \ 
;                      37 38 43 44 59 49 56 55 62 61 68  67 74  73
;
[ moleculetype ]
  DPPC  3   

[ atoms ]
;
;  nr type rnr res   atom cgnr charge 
     1  CN3  1  DPPC  CA     0  -0.2795
     2  HCN  1  DPPC  H1A    1   0.1484
     3  HCN  1  DPPC  H2A    2   0.1484
     4  HCN  1  DPPC  H3A    3   0.1484
     5  CN3  1  DPPC  CB     4  -0.2795
     6  HCN  1  DPPC  H1B    5   0.1484
     7  HCN  1  DPPC  H2B    6   0.1484
     8  HCN  1  DPPC  H3B    7   0.1484
     9  CN3  1  DPPC  CC     8  -0.2795
    10  HCN  1  DPPC  H1C    9   0.1484
    11  HCN  1  DPPC  H2C   10   0.1484
    12  HCN  1  DPPC  H3C   11   0.1484
    13  N3c  1  DPPC  N3    12   0.3372
    14  CT   1  DPPC  CD    13  -0.1311
    15  HC   1  DPPC  H1D   14   0.1484
    16  HC   1  DPPC  H2D   15   0.1484
    17  CT   1  DPPC  CE    16   0.2654
    18  HC   1  DPPC  H1E   17   0.0161
    19  HC   1  DPPC  H2E   18   0.0161
    20  OSp  1  DPPC  O7    19  -0.6096
    21  O2P  1  DPPC  O9    20  -0.9666
    22  O2P  1  DPPC  O10   21  -0.9666
    23  P    1  DPPC  P8    22   1.5572
    24  OSp  1  DPPC  O11   23  -0.6096
    25  CT   1  DPPC  CF    24   0.2654
    26  HC   1  DPPC  H1F   25   0.0161
    27  HC   1  DPPC  H2F   26   0.0161
    28  CT   1  DPPC  C13   27   0.2496
    
    ....
```

The SI provides the parameters in an itp file.  I tried using the bulk of these parameters, stopping basically at carbon labeled opls_182 (the carbon right before the ester in what I consider the "base" of the chain, i.e., not the side with N).   I used the charges as is, not rounded like christoph did...let us see if we are charge neutral or anywhere close

In [ ]:
oplsaa_mpc_new_v1 = general_forcefield.Forcefield(forcefield_files='mpc_no_impropers_v1.xml', strict=False)

In [ ]:
typed_chain_new_v1 = oplsaa_mpc_new_v1.apply(mpc, assert_improper_params=False, assert_dihedral_params=False, debug=True)

In [ ]:
sum_of_charges = 0
for atom in typed_chain_new_v1.sites:
    #print(atom.name, '\t', atom.atom_type.name,  '\t', atom.charge.v, '\t', oplsaa_mpc_new_v1.atomTypeDefinitions[atom.atom_type.name])
    sum_of_charges += atom.atom_type.charge
print(f"Total charge of molecule {sum_of_charges.v}")

The chain is still not charge neutral. However if we sum up the charges up to (and not including) opls_182, we are effectively charge neutral in the region where I adjusted charges (see data below). 

The issue seems to be that I modified opls_140 (the hydrogens attached to CH3).  These charges are substantially larger than the original OPLS forcefield (0.06).  While this works well for the methyl groups attached to nitrogen, it applies these to the methyl groups not attached to nitrogen at the end of the chain. 

This will result in a net increase of 0.2652 which almost all of the amount we are off by.  If we correc this we can easily fudge the charge on another atom to make this work (I'll just add this to the opls_182 charge to make it 0.19). 

This is a pretty simple  fix:  add in a new hydrogen atom type (opls_140b) that overrides opls_140 if the methyl carbon it is attached to is attached to nitrogen. 

```
     <Type name="opls_140" class="HC" element="H" mass="1.00800"  def="H[C;X4]" desc="alkane H" doi="10.1021/ja9621760"/>
     <Type name="opls_140b" class="HC" element="H" mass="1.00800"  def="H[C;X4](N)" desc="alkane H" overrides='opls_140' doi="10.1021/ja9621760"/>


    <Atom type="opls_140" charge="0.06" sigma="0.25" epsilon="0.12552"/>
    <Atom type="opls_140b" charge="0.1484" sigma="0.25" epsilon="0.12552"/>

```

I'll note we could accomplish the same thing this  recursion where we specifically referring to the carbon atom_type.  However, we would need two new hydrogen atom_types, because the two applicable atom_types are opls_291 and opls_292.(i.e., H[C;opls_291] and H[C;opls_292]).  If we needed the charges on hydrogen to differ based on the carbon, we might want to do it this way, or simply further specify carbon neighbors. 



Excuse the crappy cut and paste from excel below...I didn't want to waste time figuring out how to cut up the graph to my liking to sum these in an automated fashion (but probably should do that to make future setup easier). 

```
0.1484	H	opls_140
0.1484	H	opls_140
0.1484	H	opls_140
-0.2795	C	opls_291
0.1484	H	opls_140
0.1484	H	opls_140
0.1484	H	opls_140
-0.2795	C	opls_291
0.1484	H	opls_140
0.1484	H	opls_140
0.1484	H	opls_140
-0.2795	C	opls_291
0.3372	N	opls_288
-0.1311	C	opls_292
0.1484	H	opls_140
0.1484	H	opls_140
0.2654	C	opls_443
0.0161	H	opls_444
0.0161	H	opls_444
-0.6096	O	opls_442
1.5572	P	opls_440
-0.9666	O	opls_441
-0.9666	O	opls_441
-0.6096	O	opls_442
0.2654	C	opls_443
0.0161	H	opls_185
0.0161	H	opls_185
0.14	C	opls_182
0.03	H	opls_185
0.03	H	opls_185
-0.33	O	opls_467
0.51	C	opls_465
-0.43	O	opls_466
0.00	C	opls_141
-0.18	C	opls_135
0.1484	H	opls_140
0.1484	H	opls_140
0.1484	H	opls_140
-0.23	C	opls_143
0.115	H	opls_144
0.115	H	opls_144
```

## Test out the new forcefield

In [ ]:
oplsaa_mpc_new_v2 = general_forcefield.Forcefield(forcefield_files='mpc_no_impropers_v2.xml', strict=False)

In [ ]:
typed_chain_new_v2 = oplsaa_mpc_new_v2.apply(mpc, assert_improper_params=False, assert_dihedral_params=False, debug=True)

In [ ]:
sum_of_charges = 0
for atom in typed_chain_new_v2.sites:
    sum_of_charges += atom.atom_type.charge
print(f"Total charge of molecule {sum_of_charges.v}")

I'd say that is pretty close to zero.  I've also included the file mpc_v2.xml which has the impropers uncommented.


Let us fire up the visualization again and take a look at what we have. 

In [ ]:
graph = to_networkx(typed_chain_new_v2)
topology = typed_chain_new_v2

In [ ]:
#Dash Plotly representation of the molecule
app = JupyterDash(__name__)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

color_dictionary = {'"C"':"#8C92AC",'"O"':"red",'"H"':"#848482",'"N"':"blue",'"Cl"':'green'}
color_selection = []
for key,value in color_dictionary.items():
    color_selection.append({'selector': "[element = " +  str(key) + "]",
                            'style': {'background-color': str(value),'shape': 'circle'}})

layout = nx.drawing.layout.kamada_kawai_layout(graph)
elements2 = []
nodeids = {}
for i,node in enumerate(graph.nodes):
    elements2.append({'data': {'id': str(i) + ': ' + node.name, 
                               'label': node.name + '(' + f'{node.charge.v}' + ')',
                              'element': node.name,
                              'hash': node.__hash__(),
                              'index' : i},
                     'classes': '',
                     'position': {'x': layout[node][0]*500,'y': layout[node][1]*500},
                     'size': 200})
    nodeids[node] = str(i) + ': ' + node.name

for edge in graph.edges:
    elements2.append({'data': {'source': nodeids[edge[0]], 'target': nodeids[edge[1]]}, 'classes': 'red'})
       
app.layout = html.Div([
    html.P("Dash Cytoscape:"),
    dcc.Dropdown(
        id='parameter-dropdown',
        options=[
            {'label': 'Atom Types', 'value': 'atom_type'},
            {'label': 'Bond Types', 'value': 'bond_type'},
            {'label': 'Angle Types', 'value': 'angle_type'},
            {'label': 'Dihedral Types', 'value': 'dihedral_type'},

        ],
        value='atom_type'),
    html.Div([
        cyto.Cytoscape(
            id='cytoscape',
            elements= elements2,
            style={'width': '900px', 'height': '500px'},
            layout={'name':'preset'},
            stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(label)', 'width': "50%", 'height': "50%"
                }
            }
        ] + color_selection 
        )],
    style={'width': '45%', 'display': 'inline-block', 'margin-right': '20px',
          'borderBottom': 'thin lightgrey solid',
          'borderTop': 'thin lightgrey solid',
          'backgroundColor': 'rgb(250, 250, 250)',
          'vertical-align': 'top'}
    ),    
    html.Div([    
        dash_table.DataTable(
            id='table-of-parameters',
            style_as_list_view=True,
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto',
                'lineHeight': '15px'
            },
            style_table={'overflowX': 'auto','overflowY': 'auto',
                        'height': '500px', 'width': '400px'},
            style_data_conditional=[
            {
            'if': {'row_index': 'even'},
            'backgroundColor': 'rgb(248, 248, 248)'
            }],
            style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
            }
        )
    ],
    style={'width': '45%','display': 'inline-block', 'padding': '0 0'}
    )
])

@app.callback(
    Output('table-of-parameters', 'columns'),
    Output('table-of-parameters', 'data'),
    Input('parameter-dropdown','value'))
def update_datatable(input_value):
    if input_value == 'atom_type':
        df = demo_utils.atomtypes_to_datatables(graph,
                                                labels=None,atom_objects = True)
    elif input_value == 'bond_type':
        df = demo_utils.bondtypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'angle_type':
        df = demo_utils.angletypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    elif input_value == 'dihedral_type':
        df = demo_utils.dihedraltypes_to_datatables(graph,topology,labels=None,atom_objects = True)
    return([{"name": i, "id": i} for i in df.columns],
          df.to_dict('records'))

@app.callback(Output('table-of-parameters', 'style_data_conditional'),
              Input('cytoscape', 'tapNodeData'))
def displayTapNodeData(data):
    try:
        return ([{
            'if': {'filter_query': '{atom_id} = ' + str(data['hash'])},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']),
                   'column_id': ''},
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'},
            {'if': {'filter_query': '{atom1_id} = ' + str(data['hash']) +
                                ' || {atom2_id} = ' + str(data['hash']) + 
                                ' || {atom3_id} = ' + str(data['hash']) +
                                ' || {atom4_id} = ' + str(data['hash']) 
                   },
            'backgroundColor': 'tomato',
            'color': 'white'}

        ])
    except TypeError:
        return


app.run_server(mode = "inline")